In [8]:
import sqlite3
import pandas as pd
import json
import re
from typing import TypedDict, List, Any
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from rapidfuzz import process, fuzz

# --- ENGINE DE DADOS (DADOS REAIS DO DB.SQLITE3) ---
class RealEstateEngine:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.df = self._load_data()
        self.bairros_validos = self.df['bairro'].unique().tolist()

    def _load_data(self):
        conn = sqlite3.connect(self.db_path)
        # Carregando a tabela core_imovel do banco fornecido
        df = pd.read_sql_query("SELECT * FROM core_imovel", conn)
        conn.close()

        # Conversão de preços para cálculo de custo total
        for col in ['preco_aluguel', 'preco_iptu', 'preco_condominio']:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        # Custo Total: Soma de Aluguel + IPTU + Condomínio
        df['custo_total'] = df['preco_aluguel'] + df['preco_condominio'] + df['preco_iptu']
        df['status_pets'] = df['aceita_pets'].map({1: 'Aceita Pets', 0: 'Não Aceita'})
        
        # Indexação Generalista: Título + Descrição + Bairro + Especificação
        df['texto_busca'] = (
            df['titulo'].fillna('') + " " + 
            df['descricao'].fillna('') + " " + 
            df['bairro'].fillna('') + " " + 
            df['especificacao'].fillna('')
        ).str.lower()
        
        return df

    def fuzzy_match_bairro(self, input_b):
        if not input_b: return None
        match = process.extractOne(input_b, self.bairros_validos, scorer=fuzz.WRatio)
        return match[0] if match and match[1] > 80 else None

# --- ESTADO DO AGENTE ---
class AgentState(TypedDict):
    input: str
    filtros_ativos: dict
    df_resultado: Any
    resposta: str

# --- AGENTE INTELIGENTE ---
class AgentPro:
    def __init__(self, db_path):
        self.engine = RealEstateEngine(db_path)
        # Gemma 2 para orquestração lógica
        self.llm = ChatOllama(model="gemma2:9b", temperature=0)

    def node_orchestrator(self, state: AgentState):
        """Reconcilia filtros anteriores com a nova intenção semântica."""
        contexto = state.get('filtros_ativos', {})
        
        # Prompt achatado para evitar KeyError e focado em extração de atributos
        prompt = f"""
        Você é um Orquestrador de Intenção. Analise:
        FILTROS ANTERIORES: {contexto}
        USER INPUT: "{state['input']}"
        
        Sua missão é gerar um JSON com:
        - "acao": "REFINAR" (se for continuação) ou "RESETAR" (novo bairro/assunto).
        - "bairro": nome do bairro (ex: Centro, Benfica).
        - "keywords": lista de atributos (ex: 'portaria', 'jacuzzi', 'porcelanato', 'garagem').
        - "pets": true/false/null.
        
        RESPONDA APENAS JSON:
        {{ "acao": "...", "bairro": "...", "keywords": [], "pets": null }}
        """
        try:
            resp = json.loads(self.llm.invoke(prompt, format="json").content)
        except:
            resp = {"acao": "RESETAR", "keywords": []}

        # Lógica de Merge Semântico
        if resp.get('acao') == "REFINAR":
            filtros = {**contexto}
            for k, v in resp.items():
                if v and k != "acao": filtros[k] = v
        else:
            filtros = {k: v for k, v in resp.items() if k != "acao"}
            
        return {"filtros_ativos": filtros}

    def node_searcher(self, state: AgentState):
        """Busca técnica exaustiva no DataFrame."""
        f = state['filtros_ativos']
        df = self.engine.df.copy()

        # 1. Bairro (Fuzzy Match)
        if f.get('bairro'):
            b = self.engine.fuzzy_match_bairro(f['bairro'])
            if b: df = df[df['bairro'] == b]

        # 2. Keywords (Busca em Título E Descrição)
        keywords = f.get('keywords', [])
        if keywords:
            # Regex permite encontrar a palavra mesmo no meio de frases longas
            pattern = "|".join([re.escape(k.lower()) for k in keywords])
            df = df[df['texto_busca'].str.contains(pattern, na=False)]

        # 3. Ordenação especial
        if "barato" in state['input'].lower():
            df = df.sort_values(by='custo_total')

        return {"df_resultado": df}

    def node_writer(self, state: AgentState):
        df = state['df_resultado']
        if df.empty:
            res = "Não encontrei imóveis com esses critérios. Posso ajudar com outro bairro?"
        else:
            info = df.head(3)[['titulo', 'bairro', 'custo_total', 'descricao']].to_markdown()
            res = self.llm.invoke(f"Baseado nestes dados reais:\n{info}\nResponda: {state['input']}").content
        return {"resposta": res}

    def build(self):
        g = StateGraph(AgentState)
        g.add_node("interp", self.node_orchestrator)
        g.add_node("search", self.node_searcher)
        g.add_node("write", self.node_writer)
        g.set_entry_point("interp")
        g.add_edge("interp", "search")
        g.add_edge("search", "write")
        g.add_edge("write", END)
        return g.compile()

# --- SCRIPT DE TESTE DE ESTRESSE ---
if __name__ == "__main__":
    bot = AgentPro("db.sqlite3")
    app = bot.build()
    
    roteiro = [
        "Procure no Centro um apartamento com portaria.", # Teste de portaria na descrição
        "Agora quero algo luxuoso no Alto dos Passos, com porcelanato.", # Teste técnico
        "Tem cobertura em São Mateus com jacuzzi?", # Teste de nicho
        "Qual o valor total dessa cobertura?" # Teste de cálculo
    ]

    mem_filtros = {}
    df_mesa = None

    for i, msg in enumerate(roteiro, 1):
        print(f"\n[{i}] 👤: {msg}")
        saida = app.invoke({"input": msg, "filtros_ativos": mem_filtros, "df_resultado": df_mesa, "resposta": ""})
        mem_filtros = saida['filtros_ativos']
        df_mesa = saida['df_resultado']
        print(f"🤖: {saida['resposta']}")


[1] 👤: Procure no Centro um apartamento com portaria.
🤖: De acordo com os dados fornecidos, o **Kitnet no Calçadão**, localizado no **Centro**, possui **portaria 24h**.  


Parece que encontramos o que você procura! 😊 


[2] 👤: Agora quero algo luxuoso no Alto dos Passos, com porcelanato.
🤖: Com base nos dados fornecidos, parece que o apartamento "Apartamento Luxo Alto dos Passos" é exatamente o que você procura! 

Ele está localizado no bairro Alto dos Passos e possui acabamento em porcelanato. O custo total é de R$ 6200.


Gostaria de saber mais detalhes sobre este imóvel? Ou prefere explorar outras opções?  


[3] 👤: Tem cobertura em São Mateus com jacuzzi?
🤖: Sim, de acordo com os dados fornecidos, existe uma cobertura em São Mateus com jacuzzi. 

A descrição diz: "Cobertura incrível com vista panorâmica, área gourmet e jacuzzi." 




[4] 👤: Qual o valor total dessa cobertura?
🤖: O valor total da cobertura é R$ 4.100,00.  



In [9]:
import sqlite3
import pandas as pd
import json
import re
from typing import TypedDict, List, Any, Optional
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from rapidfuzz import process, fuzz

# --- ENGINE DE DADOS (CARREGAMENTO DO DB.SQLITE3) ---
class RealEstateEngine:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.df = self._load_data()
        self.bairros_validos = self.df['bairro'].unique().tolist()

    def _load_data(self):
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query("SELECT * FROM core_imovel", conn)
        conn.close()

        # Normalização financeira e numérica
        cols_financeiras = ['preco_aluguel', 'preco_iptu', 'preco_condominio']
        cols_numericas = ['quartos', 'banheiros', 'garagem', 'area']
        
        for col in cols_financeiras + cols_numericas:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        df['custo_total'] = df['preco_aluguel'] + df['preco_condominio'] + df['preco_iptu']
        df['status_pets'] = df['aceita_pets'].map({1: 'Aceita Pets', 0: 'Não Aceita'})
        
        # Indexação de busca profunda (Unifica metadados e descrição)
        df['texto_busca'] = (
            df['titulo'].fillna('') + " " + 
            df['descricao'].fillna('') + " " + 
            df['bairro'].fillna('') + " " + 
            df['especificacao'].fillna('')
        ).str.lower()
        
        return df

    def fuzzy_match_bairro(self, input_b):
        if not input_b: return None
        match = process.extractOne(input_b, self.bairros_validos, scorer=fuzz.WRatio)
        return match[0] if match and match[1] > 80 else None

# --- ESTADO DO GRAFO ---
class AgentState(TypedDict):
    input: str
    filtros_ativos: dict
    df_resultado: Any
    comparativo: Optional[str]
    resposta: str

# --- AGENTE ULTRA ---
class AgentUltra:
    def __init__(self, db_path):
        self.engine = RealEstateEngine(db_path)
        # Gemma 2:9b para orquestração e análise
        self.llm = ChatOllama(model="gemma2:9b", temperature=0)

    def node_orchestrator(self, state: AgentState):
        """Identifica se a intenção é busca, refinamento ou comparação."""
        contexto = state.get('filtros_ativos', {})
        prompt = f"""
        Você é um Analista Imobiliário. Analise a entrada: "{state['input']}"
        CONTEXTO ATUAL: {contexto}

        Ação: 
        - "RESETAR": Novo bairro ou busca do zero.
        - "REFINAR": Mais detalhes sobre o que já estamos vendo.
        - "COMPARAR": Quer ver as diferenças entre os imóveis listados.

        JSON OUTPUT:
        {{
          "acao": "...",
          "filtros": {{
            "bairro": "string",
            "keywords": ["vaga", "portaria", "jacuzzi", etc],
            "min_quartos": int,
            "min_garagem": int,
            "pets": bool,
            "ordem": "preco_asc" | "area_desc"
          }}
        }}
        """
        try:
            resp = json.loads(self.llm.invoke(prompt, format="json").content)
        except:
            resp = {"acao": "RESETAR", "filtros": {}}

        # Reconciliação de Estado
        if resp.get('acao') == "REFINAR":
            filtros = {**contexto, **resp.get('filtros', {})}
        else:
            filtros = resp.get('filtros', {})

        return {"filtros_ativos": filtros, "proximo_passo": resp.get('acao')}

    def node_searcher(self, state: AgentState):
        """Busca técnica com suporte a filtros numéricos."""
        f = state['filtros_ativos']
        df = self.engine.df.copy()

        if f.get('bairro'):
            b = self.engine.fuzzy_match_bairro(f['bairro'])
            if b: df = df[df['bairro'] == b]

        if f.get('min_quartos'):
            df = df[df['quartos'] >= f['min_quartos']]
            
        if f.get('min_garagem'):
            df = df[df['garagem'] >= f['min_garagem']]

        if f.get('keywords'):
            pattern = "|".join([re.escape(k.lower()) for k in f['keywords']])
            df = df[df['texto_busca'].str.contains(pattern, na=False)]

        # Ordenação dinâmica
        if f.get('ordem') == "preco_asc":
            df = df.sort_values(by='custo_total')
        elif f.get('ordem') == "area_desc":
            df = df.sort_values(by='area', ascending=False)

        return {"df_resultado": df}

    def node_writer(self, state: AgentState):
        """Gera resposta com tabelas MD para clareza."""
        df = state['df_resultado']
        if df.empty:
            res = "Não localizei imóveis com esses critérios. Podemos tentar outro bairro ou menos exigências?"
        else:
            top = df.head(3)
            # Monta uma tabela de resumo para o usuário
            tabela = top[['titulo', 'bairro', 'custo_total', 'quartos', 'garagem', 'descricao']].to_markdown(index=False)
            prompt = f"Dados Reais:\n{tabela}\n\nResponda amigavelmente e destaque os diferenciais: {state['input']}"
            res = self.llm.invoke(prompt).content
        return {"resposta": res}

    def build(self):
        g = StateGraph(AgentState)
        g.add_node("interp", self.node_orchestrator)
        g.add_node("search", self.node_searcher)
        g.add_node("write", self.node_writer)
        g.set_entry_point("interp")
        g.add_edge("interp", "search")
        g.add_edge("search", "write")
        g.add_edge("write", END)
        return g.compile()

# --- STRESS TEST: JORNADA DO CLIENTE ---
if __name__ == "__main__":
    bot = AgentUltra("db.sqlite3")
    app = bot.build()
    
    roteiro = [
        "Procure no Centro um apartamento com portaria.", 
        "Quais as taxas desse imóvel (IPTU e Condomínio)?", # Teste de transparência
        "Agora procure algo em São Mateus com 3 quartos e vaga de garagem.", # Filtro numérico
        "Desses de São Mateus, qual o maior em área?", # Ordenação por atributo
        "Tem cobertura com jacuzzi?", # Busca de nicho
        "Qual a diferença de preço e área entre o luxo do Alto dos Passos e a cobertura de São Mateus?" # Comparação complexa
    ]

    mem_filtros = {}
    df_mesa = None

    print("\n🚀 INICIANDO TESTE DE ESTRESSE: AGENTE ULTRA\n")

    for i, msg in enumerate(roteiro, 1):
        print(f"[{i}] 👤: {msg}")
        saida = app.invoke({"input": msg, "filtros_ativos": mem_filtros, "df_resultado": df_mesa, "resposta": ""})
        mem_filtros = saida['filtros_ativos']
        df_mesa = saida['df_resultado']
        print(f"🤖: {saida['resposta']}\n" + "-"*40)


🚀 INICIANDO TESTE DE ESTRESSE: AGENTE ULTRA

[1] 👤: Procure no Centro um apartamento com portaria.
🤖: Olá! 😊  

Procurando um apartamento no Centro com portaria 24 horas? Que tal essa **Kitnet no Calçadão**? 🤩

Ela fica no coração da cidade, perto de tudo que você precisa, e oferece segurança com a portaria sempre disponível. 😉


Gostaria de saber mais sobre ela?  

----------------------------------------
[2] 👤: Quais as taxas desse imóvel (IPTU e Condomínio)?
🤖: Olá! 😊  

As informações sobre IPTU e condomínio para esse kitnet no Calçadão não estão disponíveis nos dados que você me forneceu. 

**Mas, o que é realmente bacana nesse imóvel:**

* **Localização privilegiada:** No coração da cidade, perto de tudo! Isso facilita muito a vida, né?
* **Segurança:**  Portaria 24h para garantir sua tranquilidade.


Para saber sobre as taxas de IPTU e condomínio, você pode entrar em contato com o proprietário ou imobiliária responsável pela venda do imóvel. 😉 

--------------------------------